Created on Thu Oct 26 10:28:08 2023

@author: Santiago D'hers

Use:

- This script will help us visualize the results of labeled videos

Requirements:

- The position.csv files processed by 1-Manage_H5.py

- Geolabels, autolabels or manual labels

In [38]:
import os
from glob import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

import plotly.graph_objs as go
from plotly.subplots import make_subplots

import csv

In [39]:
# State your path:
path = r'C:\Users\dhers\OneDrive - UBA\workshop'
experiment = r'FC'

folder = os.path.join(path, experiment)
trial = 'TS'
objects = []

t_lim = None # seconds
video_fps = 12.5

In [40]:
def create_reference(folder, trial, objects):
    
    reference_path = os.path.join(folder, 'reference.csv')
    
    # Check if Reference.csv already exists
    if os.path.exists(reference_path):
        print("Reference file already exists")
        return reference_path
    
    # Get a list of all CSV files in the labels folder
    labels_files = glob(os.path.join(folder,f"{trial}/position/*position.csv"))
    labels_files = sorted(labels_files)

    # Create a new CSV file with a header 'Videos'
    with open(reference_path, 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)
        col_list = ['Video','Group'] + objects
        csv_writer.writerow(col_list)

        # Write each position file name in the 'Videos' column
        for file in labels_files:
            # Remove "_position.csv" from the file name
            clean_name = os.path.basename(file).replace(f'_position.csv', '')
            csv_writer.writerow([clean_name])

    print(f"CSV file '{reference_path}' created successfully with the list of video files.")
    
    return reference_path

In [41]:
# Lets create the reference.csv file
reference_path = create_reference(folder, trial, objects)

Reference file already exists


STOP! Go to the Reference file and complete the 'Group','Left' and 'Right' columns

In [42]:
def rename_labels(reference_path, trial):
    
    parent_dir = os.path.dirname(reference_path)
    reference = pd.read_csv(reference_path)
    
    # Create a subfolder named "final_{label_type}"
    summary_path = os.path.join(parent_dir, f'summary')

    # Check if it exists
    if os.path.exists(summary_path):
        print(f'summary folder already exists')
    else:
        os.makedirs(summary_path, exist_ok = True)
    
    group_list = []
    
    # Iterate through each row in the table
    for index, row in reference.iterrows():
        
        video_name = row['Video']
        group = row['Group']

        group_path = os.path.join(summary_path, group)
        os.makedirs(group_path, exist_ok = True)

        # Find the old file path
        old_path = os.path.join(parent_dir, trial, 'movement', f'{video_name}_movement.csv')
            
        # Read the CSV file into a DataFrame
        df = pd.read_csv(old_path)

        # Here you can modify it as you need...

        # Create the new file path
        new_name = f'{video_name}_summary.csv'
        new_path = os.path.join(group_path, new_name)
    
        # Save the modified DataFrame to a new CSV file
        df.to_csv(new_path, index=False)
        
        group_list.append(group)
    
        print(f'Renamed and saved: {new_path}')
    
    group_list = sorted(list(set(group_list)))
        
    return summary_path, group_list

In [ ]:
# Lets rename the labels
summary_path, groups = rename_labels(reference_path, trial)

In [44]:
subfolders = summary_path.split(os.path.sep)
subfolders

['C:', 'Users', 'dhers', 'OneDrive - UBA', 'workshop', 'FC', 'summary']

In [55]:
def plot_groups(path, groups, fps = 30):
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    for group in groups:
        for filename in os.listdir(os.path.join(path, group)):
            
            file_path = os.path.join(path, group, filename)
            df = pd.read_csv(file_path)
            
            df["nose_dist_cumsum"] = df["nose_dist"].cumsum()
            df["body_dist_cumsum"] = df["body_dist"].cumsum()

            df["freezing_cumsum"] = df["freezing"].cumsum()
            
            dfs.append(df)
                
        n = len(dfs) # We find the number of mice to calculate the standard error as std/sqrt(n)
        se = np.sqrt(n)
        
        # Find the minimum length of all files
        min_length = min([len(df) for df in dfs])

        trunc_dfs = [df.iloc[:min_length] for df in dfs]

        # Concatenate the list of DataFrames into one DataFrame
        all_dfs = pd.concat(trunc_dfs, ignore_index=True)

        # Calculate the mean and standard deviation of cumulative sums for each frame
        df = all_dfs.groupby('Frame').agg(['mean', 'std']).reset_index()
            
        # Create a single figure
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        df['time_seconds'] = df['Frame'] / fps
            
        # Distance covered
        axes[0].plot(df['time_seconds'], df[("nose_dist_cumsum" ,'mean')], label = "nose distance")
        axes[0].fill_between(df['time_seconds'], df[("nose_dist_cumsum" ,'mean')] - df[("nose_dist_cumsum", 'std')]/se, df[("nose_dist_cumsum" ,'mean')] + df[("nose_dist_cumsum" ,'std')]/se, alpha=0.2)
        axes[0].plot(df['time_seconds'], df[("body_dist_cumsum" ,'mean')], label = "body distance")
        axes[0].fill_between(df['time_seconds'], df[("body_dist_cumsum" ,'mean')] - df[("body_dist_cumsum", 'std')]/se, df[("body_dist_cumsum" ,'mean')] + df[("body_dist_cumsum" ,'std')]/se, alpha=0.2)
        axes[0].set_xlabel('Time (s)')
        axes[0].set_xticks([0, 60, 120, 180, 240, 300])
        axes[0].set_ylabel('Distance (m)')
        axes[0].set_title('Distance Traveled in Habituation')
        axes[0].legend(loc='upper left', fancybox=True, shadow=True)
        axes[0].grid(True)

        # Freezing
        axes[1].plot(df['time_seconds'], df[("freezing_cumsum" ,'mean')], label = "freezing")
        axes[1].fill_between(df['time_seconds'], df[("freezing_cumsum" ,'mean')] - df[("freezing_cumsum", 'std')]/se, df[("freezing_cumsum" ,'mean')] + df[("freezing_cumsum" ,'std')]/se, alpha=0.2)
        axes[1].set_xlabel('Time (s)')
        axes[1].set_xticks([0, 60, 120, 180, 240, 300])
        axes[1].set_ylabel('Time (s)')
        axes[1].set_title('Distance Traveled in Habituation')
        axes[1].legend(loc='upper left', fancybox=True, shadow=True)
        axes[1].grid(True)
        
        plt.suptitle(f"Analysis of {group}", y=0.98)  # Add DataFrame name as the overall title
        plt.tight_layout()
        plt.savefig(os.path.join(path, f"{group}.png"))
        plt.show()

In [ ]:
plot_groups(summary_path, groups, fps = video_fps)

In [67]:
def plot_experiment(path, groups, fps = 30):

    # Create a single figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    bxplt_positions = list(range(1, len(groups) + 1))
    
    # Initialize an empty list to store DataFrames
    dfs = []
    
    for i, group in enumerate(groups):
        
        bxplt = []

        for filename in os.listdir(os.path.join(path, group)):
            
            file_path = os.path.join(path, group, filename)
            df = pd.read_csv(file_path)
            
            df["nose_dist_cumsum"] = df["nose_dist"].cumsum()
            df["body_dist_cumsum"] = df["body_dist"].cumsum()

            df["freezing_cumsum"] = (df["freezing"]/fps).cumsum()
            
            dfs.append(df)

            bxplt.append(df.loc[df.index[-1], "freezing_cumsum"])
                
        n = len(dfs) # We find the number of mice to calculate the standard error as std/sqrt(n)
        se = np.sqrt(n)
        
        # Find the minimum length of all files
        min_length = min([len(df) for df in dfs])

        trunc_dfs = [df.iloc[:min_length] for df in dfs]

        # Concatenate the list of DataFrames into one DataFrame
        all_dfs = pd.concat(trunc_dfs, ignore_index=True)

        # Calculate the mean and standard deviation of cumulative sums for each frame
        df = all_dfs.groupby('Frame').agg(['mean', 'std']).reset_index()

        bxplt = pd.DataFrame(bxplt)
        
        df['time_seconds'] = df['Frame'] / fps
            
        # Distance covered
        axes[0].plot(df['time_seconds'], df[("body_dist_cumsum" ,'mean')], label = f"{group} body distance")
        axes[0].fill_between(df['time_seconds'], df[("body_dist_cumsum" ,'mean')] - df[("body_dist_cumsum", 'std')]/se, df[("body_dist_cumsum" ,'mean')] + df[("body_dist_cumsum" ,'std')]/se, alpha=0.2)
        axes[0].set_xlabel('Time (s)')
        axes[0].set_xticks([0, 60, 120, 180, 240, 300])
        axes[0].set_ylabel('Distance (m)')
        axes[0].set_title('Distance Traveled')
        axes[0].legend(loc='upper left', fancybox=True, shadow=True)
        axes[0].grid(True)

        # Freezing
        axes[1].plot(df['time_seconds'], df[("freezing_cumsum" ,'mean')], label = f"{group} freezing")
        axes[1].fill_between(df['time_seconds'], df[("freezing_cumsum" ,'mean')] - df[("freezing_cumsum", 'std')]/se, df[("freezing_cumsum" ,'mean')] + df[("freezing_cumsum" ,'std')]/se, alpha=0.2)
        axes[1].set_xlabel('Time (s)')
        axes[1].set_xticks([0, 60, 120, 180, 240, 300])
        axes[1].set_ylabel('Time (s)')
        axes[1].set_title('Time spent freezing')
        axes[1].legend(loc='upper left', fancybox=True, shadow=True)
        axes[1].grid(True)

        # Boxplot
        axes[2].boxplot(bxplt[0], positions=[bxplt_positions[i]], tick_labels = [f'{group}'])
        
        # Replace boxplots with scatter plots with jitter
        jitter_amount = 0.05  # Adjust the jitter amount as needed
        axes[2].scatter([i + 1 + np.random.uniform(-jitter_amount, jitter_amount) for _ in range(len(bxplt[0]))], bxplt[0], alpha=0.7, label=f'{group}')
        
        # axes[2].axhline(y=0, color='black', linestyle='--', linewidth = 2)
        axes[2].set_ylabel('Time (s)')
        axes[2].set_title('Total freezing')
        
    plt.suptitle(f"Analysis of {experiment}", y=0.98)  # Add DataFrame name as the overall title
    plt.tight_layout()
    plt.savefig(os.path.join(path, f"{experiment}.png"))
    plt.show()

In [ ]:
plot_experiment(summary_path, groups, fps=video_fps)